In [9]:
import os
import cv2
import collections
import time
import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, train_test_split

import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
import configparser
import argparse
import wandb

# Catalyst is amazing.
from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst.dl.runner import SupervisedRunner
# from catalyst.dl.runner import SupervisedWandbRunner as SupervisedRunner
from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback

# PyTorch made my work much much easier.
import segmentation_models_pytorch as smp
from dataloader import SegmentationDataset, SegmentationDatasetTest
from augmentations import get_training_augmentation, get_validation_augmentation, get_preprocessing

from utils import *
import pickle

sub = pd.read_csv(f'../input/sample_submission.csv')
sub['label'] = sub['Image_Label'].apply(lambda x: x.split('_')[1])
sub['im_id'] = sub['Image_Label'].apply(lambda x: x.split('_')[0])
test_ids = sub['Image_Label'].apply(lambda x: x.split('_')[0]).drop_duplicates().values

model_path = 'Data/kaggle-andrew-651-model.pth'
bs = 2
num_workers = 0
encoder = 'efficientnet-b2'
model, preprocessing_fn = get_model(encoder)

runner = SupervisedRunner()
encoded_pixels = []

def get_validation_augmentation():
    test_transform = [
        albu.Resize(320, 640),
        albu.CLAHE(p=1),
    ]
    return albu.Compose(test_transform)


test_dataset = SegmentationDatasetTest(test_ids, 
                                        transforms=get_validation_augmentation(), 
                                        preprocessing=get_preprocessing(preprocessing_fn),
                                        img_db="../input/test_images_525/test_images_525")
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=0)
loaders = {"test": test_loader}

# runner.infer(
#     model=model.to('cpu'),
#     loaders=loaders,
#     callbacks=[
#         CheckpointCallback(
#             resume=model_path),
#         InferCallback()
#     ],
# )

Training on unet architecture with efficientnet-b2 encoder


In [10]:
for batch in test_loader:
    print(batch.size())
    output = model.forward(batch)
    print(output.size())
    break

torch.Size([2, 3, 525, 640])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 64 and 65 in dimension 2 at /pytorch/aten/src/TH/generic/THTensor.cpp:689